In [1]:
!pip install google-generativeai

In [2]:
import google.generativeai as genai

genai.configure(api_key="use your own")

for m in genai.list_models():
    print(m.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pr

In [3]:
model = genai.GenerativeModel(model_name="gemini-2.0-flash")

In [4]:
import tensorflow_datasets as tfds

ds_test = tfds.load('imdb_reviews', split='test[:100]', as_supervised=True)

textes = []
labels = []

for text, label in tfds.as_numpy(ds_test):
    textes.append(text.decode('utf-8'))
    labels.append(int(label))  

2025-05-06 20:26:19.634757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746563179.851208      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746563179.915613      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.YXZIOK_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.YXZIOK_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2025-05-06 20:27:40.003650: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
import time
pred_gemini=[]
explications=[]
for opinion in textes :
    response = model.generate_content(f"Evaluate this movie review : '{opinion}'. Respond with 0 if the review is negative, 1 if it is positive. Respond only with 0 or 1, nothing else. Then provide a short explanation as to why you choose negative or positive.")
    print(response.text)
    pred_gemini.append(int(response.text[0]))
    explications.append(response.text)
    time.sleep(5)

1
Explanation: The review uses overwhelmingly positive language such as "absolutely amazing," "flawless film-making," "assured and professional," and praises Tukel's talent and the film's composition and performances. The comparison to films that launched careers and the statement "DING-A-LING-LESS is a film whose time has come" further solidify the positive assessment.

1

Explanation: The review uses overwhelmingly positive language, praising the film's acting, narrative, theme, and its place within Bunuel's filmography. It calls the film "remarkable" and urges viewers to seek it out.

0
The review is overwhelmingly negative. The reviewer uses strong language like "ruined the parody genre," "disaster," "laughless bile," and "worst outings of 2008." While acknowledging it might be slightly better than "Meet the Spartans" and "Epic Movie," the overall tone is highly critical and recommends avoiding the movie.

0
The review is overwhelmingly negative. The reviewer uses phrases like "maw

In [6]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(labels, pred_gemini)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.91


In [7]:
response = model.generate_content(f"Here is a list of explanations as to why 100 movie reviews were classified as they were : {explications}. Make a list of 10 bullet points of the main reasons why some reviews were classified as negative, and do the same for the positive ones but do not provide an explanation, just a reason for each bullet point")
print(response.text)

Okay, here are 10 bullet points summarizing the main reasons for negative and positive movie review classifications, as gleaned from the provided dataset:

**Reasons for Negative Classifications:**

*   Overwhelmingly negative language (e.g., "awful," "terrible," "waste of time").
*   Criticism of acting quality.
*   Criticism of plot or story structure.
*   Poor special effects.
*   Lack of humor or failed attempts at humor.
*   Disappointment with character development.
*   Comparison to superior films highlighting the reviewed film's shortcomings.
*   Low ratings/scores.
*   Explicit statements of dislike or recommendations against watching the film.
*   General sense of boredom or lack of engagement.

**Reasons for Positive Classifications:**

*   Positive language
*   Praise for acting
*   Praise for the plot or story
*   High rating
*   Explicit recommendation to watch the film
*   Enjoyment and satisfaction
*   Praise for the film's atmosphere
*   Praise for the directing
*   Pr

# Test avec un meilleur prompt généré avec GPT-4o

In [8]:
pred_gemini_best_prompt=[]
explications_best_prompt=[]
for opinion in textes :
    response = model.generate_content(f"Evaluate the sentiment of the following movie review: '{opinion}'.Respond with 0 if the review is negative, 1 if it is positive. Respond with the number on the first line only. Then, on the next line, provide a brief explanation summarizing the key sentiment cues that led to your decision. Focus only on the content of the review and avoid speculation.")
    print(response.text)
    pred_gemini_best_prompt.append(int(response.text[0]))
    explications_best_prompt.append(response.text)
    time.sleep(5)

1
The review is overwhelmingly positive, using phrases like "absolutely amazing," "flawless film-making," "assured and professional," "laughed this hard," "talent is considerable," "beautifully composed," and "performances all around are solid." It also favorably compares the film to career-making works of established directors.

1
The review uses overtly positive language such as "blackly comic tale", "deeply humanist fable", "invaluable talent", "surpasses many of Bunuel's previous Mexican films", "portrayed more sympathetically", "so emotional", "remarkable film", "one of Bunuel's most moving films", and "certainly near the top of the list of Bunuel's total filmic output." It also praises the acting as "excellent."

0
The review contains numerous negative sentiment cues, including calling the movie a "disaster," "laughless bile," and one of the "worst outings of 2008." It also criticizes the movie for ruining the parody genre and being comparable to other poorly received films like 

In [9]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(labels, pred_gemini_best_prompt)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.91


In [10]:
response = model.generate_content(f"Here is a list of explanations as to why 100 movie reviews were classified as they were : {explications_best_prompt}. Make a list of 10 bullet points of the main reasons why some reviews were classified as negative, and do the same for the positive ones but do not provide an explanation, just a reason for each bullet point")
print(response.text)

Okay, here are the bullet points of reasons for negative and positive classifications, extracted from the provided text:

**Reasons for Negative Classifications:**

*   Explicitly negative language (e.g., "awful," "hated," "waste of time")
*   Criticism of acting quality
*   Poor writing, directing, or editing
*   Unoriginal or cliché plot
*   Disappointment or unmet expectations
*   Technical flaws (e.g., special effects, CGI)
*   Sarcasm used to highlight flaws
*   Lack of humor or entertainment value
*   Characters are unlikeable or poorly developed
*   Failure to meet potential or genre expectations

**Reasons for Positive Classifications:**

*   Overwhelmingly positive language
*   Praise for acting
*   Engaging and emotionally resonant story
*   Realistic portrayal
*   Excellent technical aspects
*   Unique and brilliant
*   Entertaining and humorous
*   Thought-provoking
*   Positive impact on reviewer
*   High rating

